# 確率と統計の入門
このノートブックでは、以前に議論したいくつかの概念を試してみます。確率と統計の多くの概念は、Pythonの主要なデータ処理ライブラリである`numpy`や`pandas`などに豊富に含まれています。


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## 確率変数と分布
0から9までの一様分布から30個の値のサンプルを抽出するところから始めましょう。平均と分散も計算します。


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

サンプル内にいくつの異なる値があるかを視覚的に推定するために、**ヒストグラム**をプロットすることができます:


In [ ]:
plt.hist(sample)
plt.show()

## 実データの分析

平均値と分散は実世界のデータを分析する際に非常に重要です。[SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights) から野球選手のデータを読み込みましょう。


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> ここではデータ分析のために[**Pandas**](https://pandas.pydata.org/)というパッケージを使用しています。このコースの後半でPandasおよびPythonでのデータ操作について詳しく説明します。

年齢、身長、体重の平均値を計算してみましょう:


In [ ]:
df[['Age','Height','Weight']].mean()

では、身長に注目して、標準偏差と分散を計算しましょう：


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

平均に加えて、中央値と四分位数を見ることも理にかなっています。これらは**箱ひげ図**を使って視覚化できます：


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

データセットのサブセット、例えばプレイヤーの役割ごとにグループ化したものの箱ひげ図も作成できます。


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **注意**: この図は、平均的に一塁手の身長が二塁手の身長より高いことを示唆しています。後ほど、この仮説をより正式に検証する方法や、データが統計的に有意であることを示す方法を学びます。

年齢、身長、体重はすべて連続型の確率変数です。これらの分布はどのようになっていると思いますか？調べる良い方法は、値のヒストグラムをプロットすることです：


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## 正規分布

実際のデータと同じ平均と分散を持つ正規分布に従う人工的な体重のサンプルを作成しましょう:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

実際の多くの値は正規分布しているため、サンプルデータを生成する際に一様乱数生成器を使用すべきではありません。以下は、一様分布（`np.random.rand`で生成）を使って体重を生成しようとした場合に起こることです：


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## 信頼区間

野球選手の体重と身長の信頼区間を計算してみましょう。コードは[このstackoverflowの議論](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data)から使用します：


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## 仮説検定

ベースボール選手のデータセットで異なる役割を見てみましょう:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

ファーストベースマンがセカンドベースマンよりも背が高いという仮説を検証してみましょう。これを行う最も簡単な方法は、信頼区間を検定することです。


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

区間が重なっていないことがわかります。

仮説を証明するために統計的により正確な方法は、**Studentのt検定**を使用することです：


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

`ttest_ind` 関数によって返される2つの値は以下の通りです:
* p値は、2つの分布が同じ平均を持つ確率と考えることができます。今回の場合、この値は非常に低く、ファーストベースマンの方が身長が高いという強い証拠を示しています。
* t値は、t検定で使用される正規化された平均差の中間値であり、与えられた信頼度に対する閾値と比較されます。


## 中央極限定理による正規分布のシミュレーション

Pythonの擬似乱数生成器は一様分布を提供するように設計されています。正規分布の生成器を作成したい場合、中央極限定理を使用できます。正規分布に従う値を得るために、一様に生成されたサンプルの平均を計算するだけです。


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## 相関とイービル・ベースボール・コープ

相関はデータ列間の関係を見つけることを可能にします。おもちゃの例として、悪のベースボール会社が選手の身長に応じて報酬を支払っていると仮定しましょう — 選手が背が高いほど、もらえるお金も多くなります。基本給は1000ドルで、身長に応じて0ドルから100ドルの追加ボーナスがあります。実際のMLB選手のデータを使い、架空の給与を計算します：


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

それではこれらのシーケンスの共分散と相関を計算しましょう。`np.cov` はいわゆる **共分散行列** を返します。これは共分散を複数の変数に拡張したものです。共分散行列 $M$ の要素 $M_{ij}$ は入力変数 $X_i$ と $X_j$ の共分散であり、対角要素 $M_{ii}$ は $X_i$ の分散です。同様に、`np.corrcoef` は **相関行列** を返します。


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

相関係数が1であるということは、2つの変数の間に強い**線形関係**があることを意味します。片方の値をもう片方に対してプロットすることで、線形関係を視覚的に確認できます。


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

関係が線形でない場合に何が起こるか見てみましょう。私たちの会社が身長と給与の明らかな線形依存性を隠し、式に `sin` のような非線形性を導入したとしましょう:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

この場合、相関はやや小さくなりますが、それでもかなり高いです。さて、この関係をさらに分かりにくくするために、給料にランダムな変数を加えていくつかのランダム性を追加したいかもしれません。何が起こるか見てみましょう：


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> なぜ点がこのように縦に並ぶか予想できますか？

私たちは、給与のような人工的に設計された概念と観測された変数*身長*との相関を観察しました。では、身長や体重のような2つの観測変数も相関するか見てみましょう：


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

残念ながら、結果は得られず、奇妙な `nan` の値だけが表示されました。これは、シリーズ内のいくつかの値が未定義で `nan` として表されているためであり、そのため計算結果も未定義になるからです。行列を見ると、`Weight` が問題のある列であることがわかります。なぜなら、`Height` の値同士の自己相関が計算されているからです。

> この例は、**データ準備**と**クリーニング**の重要性を示しています。適切なデータがなければ、何も計算できません。

`fillna` メソッドを使って欠損値を埋め、相関を計算してみましょう。


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

実際に相関は存在しますが、我々の人工的な例ほど強くはありません。実際、一方の値をもう一方の値に対して散布図で見ると、その関係ははるかに分かりにくいでしょう：


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## 結論

このノートブックでは、統計関数を計算するためにデータに対して基本的な操作を行う方法を学びました。数学と統計の確かな手法を使っていくつかの仮説を検証する方法や、データサンプルから任意の変数の信頼区間を計算する方法がわかりました。


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免責事項**：  
本書類はAI翻訳サービス「Co-op Translator」（https://github.com/Azure/co-op-translator）を使用して翻訳されました。正確性の確保に努めておりますが、自動翻訳には誤りや不正確な表現が含まれる場合があります。原文の言語で記載されたオリジナル文書が正式な情報源となります。重要な情報については、専門の翻訳者による翻訳を推奨します。本翻訳の利用により生じた誤解や解釈の違いに関して、当社は一切の責任を負いません。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
